In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as spi

In [3]:
# Initial conditions
p0 = 18000 # km
e0 = 0.12
i0 = np.radians(45) 
raan0 = np.radians(30) 
om0 = 0 
truan0 = 0 

# Target
pt = 25000 # km
et = 0
it = np.radians(50) 
raant = np.radians(20) 

In [ ]:
# phi function, correlates actual and target orbits
def phi(z: np.array):
    return np.array([
        z[0] - pt,
        z[1]**2 + z[2]**2 - et**2,
        z[3]**2 + z[4]**2 - np.tan(it/2)**2,
        z[4]/(np.sqrt(z[3]**2 + z[4]**2) + z[3]) - np.tan(raant/2)
    ])

# dphi/dz
def dphi(z: np.array ):
    return np.array([
        [1, 0, 0, 0, 0],
        [0, 2*z[1], 2*z[2], 0, 0],
        [0, 0, 0, 2*z[3], 2*z[4]],
        [0, 0, 0, -z[4]/(z[3]*(np.sqrt(z[3]**2 + z[4]**2) + z[3]) + z[4]**2), z[3]/(z[3]**2 + z[4]**2 + z[3]*np.sqrt(z[3]**2 + z[4]**2))]
    ])

# b = G.T* dphi.T *K*phi

In [12]:
# Weights
k1 = 1
k2 = 10000
k3 = 1
k4 = 1

K = np.diag([k1,k2,k3,k4])

# Propulsion parameters
ut_max = 10**(-4)*9.81*10**(-3) # km/s?
c = 30 # km/s

# Canonical units
DU = 6378 # Re = 6378 km
TU = 806.8 # 806.8 s

# Some constants
mu = 398600.4 # km^3/s^2
J2 = 1.082*10**(-3)

In [ ]:
# z_dot = G(z, mu)*(u_t/z[6] + a_p(t))
def G(z:np.array, x6, mu):
    eta = 1+ z[1]*np.cos(x6) + z[2]*np.sin(x6)
    return np.sqrt(z[0]/mu)*np.array([
        [0, 2*z[0]/eta, 0]
        [np.sin(x6), ((eta + 1)*np.cos(x6) + z[1])/eta, (-z[3]*np.sin(x6) -z[4]*np.cos(x6))*z[2]/eta],
        [-np.cos(x6), ((eta + 1)*np.sin(x6) + z[2])/eta, (z[3]*np.cos(x6) -z[4]*np.sin(x6))*z[1]/eta]
        [0,0, (1+ z[3]**2 + z[4]**2)*np.cos(x6)/(2*eta)]
        [0,0, (1+ z[3]**2 + z[4]**2)*np.sin(x6)/(2*eta)]
    ])

# z[5]_dot = F(z, mu)*(u_t/z[6] + a_p(t)).dot([0,0,1])
def F(z:np.array, mu):
    return np.sqrt(mu/z[0]**3)*(1 + z[1]*np.cos(z[5]) + z[2]*np.sin(z[5]))**2 + np.sqrt(z[0]/mu)*(z[3]*np.sin(z[5]) - z[4]*np.cos(z[5]))/(1 + z[1]*np.cos(z[5]) + z[2]*np.sin(z[5]))

# z[6]_dot = - u_t/c


def a_p(z,x6):
    # Compute inclination from z
    i = 2*np.arctan(np.sqrt(z[3]**2 + z[4]**2))
    # Compute argument of latitude from z
    theta_t = x6 - 2*np.atan(z[4]/(np.sqrt(z[3]**2 + z[4]**2) + z[3])) # x6 - raan
    # Compute position in orbit 
    r = z[0]/(1 + z[1]*np.cos(x6) + z[2]*np.sin(x6))


    fac = 3*J2*(DU**3/TU**2)*DU**2/r**4
    # Compute perturbation
    return np.array([fac*((np.sin(i)**2)*np.sin(theta_t)**2 -1)/2,  # r
                     -fac*np.sin(i)**2*np.cos(theta_t)*np.sin(theta_t), # theta 
                     -fac*np.sin(2*i)*np.cos(theta_t)*np.cos(i) # h
                     ])       

In [ ]:
l0 = e0*np.cos(raan0 + om0)
m0 = e0*np.sin(raan0 + om0)
n0 = np.tan(i0/2)*np.cos(raan0)
s0 = np.tan(i0/2)*np.sin(raan0)

q0 = raan0 + om0 + truan0

z0 = np.array([p0, l0, m0, n0, s0])
x6_0 = q0
x7_0 = 0


In [ ]:
def ode_integrate(t, z_aug):
    z = z_aug[:5]
    x6 = z_aug[5]
    x7 = z_aug[6]
    

    # Compute G matrix
    G_temp = G(z,x6, mu)

    # Get b = G.T * dphi.T * K * phi
    b = G_temp.T@dphi(z).T@K@phi(z)
    
    a_pert = a_p(z, x6)
    
    ########################################################
    # Check here if eclipse or not, if eclipse, set u_t = 0
    # else compute optimal u_t
    ########################################################
    
    # Compute optimal u_t
    temp = x7*(b + a_pert)
    
    u_t_opt =  - temp
    if np.abs(u_t_opt) > ut_max:
        u_t_opt = -ut_max*(temp/np.linalg.norm(temp))

    # Now we can compute the dynamics
    z_dot = G_temp@(u_t_opt/x7 + a_pert)
    x6_dot = np.sqrt(mu/z[0]**3)*(1 + z[1]*np.cos(x6) + z[2]*np.sin(x6))**2  \
            + np.sqrt(z[0]/mu)*(z[3]*np.sin(x6) - z[4]*np.cos(x6))/(1 + z[1]*np.cos(x6) + z[2]*np.sin(x6))*(u_t_opt[2]/x7 + a_pert[2])
    x7_dot = -np.linalg.norm(u_t_opt)/c
    
    return np.concatenate([z_dot, x6_dot, x7_dot])